<a href="https://colab.research.google.com/github/ramapathapati/hds5210-2023/blob/main/Copy_of_midterm_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [3]:
import requests

def priest(sex: str, age: int, respiratory_rate: int, oxygen_saturation: float, heart_rate: int, systolic_bp: int, temperature: float, alertness: str, inspired_oxygen: str, performance_status: str) -> float:
    try:
        # Validate the input values.
        sex = sex.lower()  # Convert to lowercase
        alertness = alertness.lower()  # Convert to lowercase
        inspired_oxygen = inspired_oxygen.lower()  # Convert to lowercase
        performance_status = performance_status.lower()  # Convert to lowercase

        if sex not in {"male", "female"}:
            raise ValueError(f"Invalid sex value: {sex}")
        if age < 0:
            raise ValueError(f"Invalid age value: {age}")
        if respiratory_rate < 0:
            raise ValueError(f"Invalid respiratory rate value: {respiratory_rate}")
        if oxygen_saturation < 0 or oxygen_saturation > 1:
            raise ValueError(f"Invalid oxygen saturation value: {oxygen_saturation}")
        if heart_rate < 0:
            raise ValueError(f"Invalid heart rate value: {heart_rate}")
        if systolic_bp < 0:
            raise ValueError(f"Invalid systolic BP value: {systolic_bp}")
        if temperature < 0:
            raise ValueError(f"Invalid temperature value: {temperature}")
        if alertness not in {"alert", "confused or not alert"}:
            raise ValueError(f"Invalid alertness value: {alertness}")
        if inspired_oxygen not in {"air", "supplemental oxygen"}:
            raise ValueError(f"Invalid inspired oxygen value: {inspired_oxygen}")
        if performance_status not in {
            "unrestricted normal activity",
            "limited strenuous activity, can do light activity",
            "limited activity, can self-care",
            "limited self-care",
        }:
            raise ValueError(f"Invalid performance status value: {performance_status}")

        # Calculate the PRIEST score.
        score = 0

        # Sex.
        if sex == "male":
            score += 1

        # Age.
        if age >= 50 and age < 66:
            score += 2
        elif age >= 66 and age < 80:
            score += 3
        elif age >= 80:
            score += 4

        # Respiratory rate.
        if respiratory_rate >= 21:
            score += 2
        elif respiratory_rate >= 12:
            score += 1

        # Oxygen saturation.
        if oxygen_saturation < 92:
            score += 3
        elif oxygen_saturation < 94:
            score += 2
        elif oxygen_saturation < 95:
            score += 1

        # Heart rate.
        if heart_rate >= 110:
            score += 2
        elif heart_rate >= 91:
            score += 1

        # Systolic BP.
        if systolic_bp < 90:
            score += 2
        elif systolic_bp < 100:
            score += 1

        # Temperature.
        if temperature >= 38.5:
            score += 1

        # Alertness.
        if alertness != "alert":
            score += 0

        if performance_status == "Unrestricted normal activity":
            score += 0

        elif performance_status == "Limited strenuous activity, can do light activity":
            score += 2

        elif performance_status == "Limited activity, can self-care":
            score += 3

        elif performance_status == "Limited self-care":
            score += 4

        total_score = score / 20.0

        if 17 <= total_score <= 25:
            return 0.59
        elif total_score >= 26:
            return 0.99
        else:
            return total_score
    except ValueError:
        return None

priest_score = priest("male", 65, 24, 0.90, 110, 140, 38.5, "alert", "supplemental oxygen", "Unrestricted normal activity")
if priest_score is not None:
    print(f"The PRIEST Clinical Severity Score is: {priest_score}")
else:
    print("The PRIEST Clinical Severity Score is: NONE")

The PRIEST Clinical Severity Score is: 0.55


In [4]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [5]:

def find_hospital(age: int, sex: str, risk: float) -> str:
    """
    Calls a REST web service to find the hospital for a patient based on age, sex, and risk percentage.

    Args:
        age (int): The age of the patient.
        sex (str): The sex of the patient (either 'male' or 'female').
        risk (float): The risk percentage of the patient.

    Returns:
        str: The name of the hospital where the patient should be sent, or None if an error occurs.

    Example:
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(25, 'female', 0.2)
    'Eastside General Hospital'
    >>> find_hospital(60, 'male', 0.8)
    'Northwest Senior Medical Center'
    """
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd"
    url = f"{base_url}?age={age}&sex={sex.lower()}&risk_pct={risk}"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data.get("hospital")
        else:
            return None
    except Exception as e:
        return None

# Test cases
print(find_hospital(40, 'male', 0.1))
print(find_hospital(25, 'female', 0.2))
print(find_hospital(60, 'male', 0.8))



JSON_URL = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"


Southwest Hospital and Medical Center
Select Specialty Hospital - Northeast Atlanta
Emory Dunwoody Medical Center


In [6]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(25, 'female', 0.2)
Expecting:
    'Eastside General Hospital'
**********************************************************************
File "__main__", line 16, in NoName
Failed example:
    find_hospital(25, 'female', 0.2)
Expected:
    'Eastside General Hospital'
Got:
    'Select Specialty Hospital - Northeast Atlanta'
Trying:
    find_hospital(60, 'male', 0.8)
Expecting:
    'Northwest Senior Medical Center'



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



**********************************************************************
File "__main__", line 18, in NoName
Failed example:
    find_hospital(60, 'male', 0.8)
Expected:
    'Northwest Senior Medical Center'
Got:
    'Emory Dunwoody Medical Center'


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [7]:
def get_address(hospital_name):
    print(f"Searching for hospital: {hospital_name}")
    try:

        with requests.get(JSON_URL) as response:
            hospital_data = response.json()

        if hospital_name in hospital_data:
            print(f"Found hospital: {hospital_name}")
            return hospital_data[hospital_name]["ADDRESS"]

        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Test cases
address = get_address("WESLEY WOODS GERIATRIC HOSPITAL")
print(f"Address: {address}")

address = get_address("JEFFERSON HOSPITAL")
print(f"Address: {address}")

address = get_address("PIEDMONT HOSPITAL")
print(f"Address: {address}")

address = get_address("MILLER COUNTY HOSPITAL")
print(f"Address: {address}")

address = get_address("FAIRVIEW PARK HOSPITAL")
print(f"Address: {address}")

import requests



Searching for hospital: WESLEY WOODS GERIATRIC HOSPITAL
Found hospital: WESLEY WOODS GERIATRIC HOSPITAL
Address: 1821 CLIFTON ROAD, NE
Searching for hospital: JEFFERSON HOSPITAL
Found hospital: JEFFERSON HOSPITAL
Address: 1067 PEACHTREE STREET
Searching for hospital: PIEDMONT HOSPITAL
Found hospital: PIEDMONT HOSPITAL
Address: 1968 PEACHTREE ROAD NW
Searching for hospital: MILLER COUNTY HOSPITAL
Found hospital: MILLER COUNTY HOSPITAL
Address: 209 NORTH CUTHBERT STREET
Searching for hospital: FAIRVIEW PARK HOSPITAL
Found hospital: FAIRVIEW PARK HOSPITAL
Address: 200 INDUSTRIAL BOULEVARD


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [8]:

def process_people(file_url):
    try:

        print("Fetching the people.psv file...")
        response = requests.get(file_url)

        if response.status_code != 200:
            print(f"Failed to retrieve the people.psv file. Status code: {response.status_code}")
            return None

        content = response.text

        lines = content.split('\n')
        data = []

        print(f"Processing {len(lines)} lines...")

        for line in lines:
            fields = line.strip().split('|')
            if len(fields) == 10:

                patient_number = fields[0]
                sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = fields[1:]

                score = priest(sex, int(age), int(breath), float(o2sat), int(heart), int(systolic), float(temp), alertness, inspired, status)

                if score is not None:

                    hospital = find_hospital(sex, int(age), score)

                    address = get_address(hospital)

                    patient_info = {
                        "patient_number": patient_number,
                        "sex": sex,
                        "age": int(age),
                        "breath": int(breath),
                        "o2sat": float(o2sat),
                        "heart": int(heart),
                        "systolic": int(systolic),
                        "temp": float(temp),
                        "alertness": alertness,
                        "inspired": inspired,
                        "status": status,
                        "hospital": hospital,
                        "address": address,
                    }

                    data.append(patient_info)

        print(f"Processed {len(data)} records.")

        return data
    except Exception as e:
        print(f"Error: {e}")
        return None



## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [9]:
import requests
import json

json_url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"

try:
    response = requests.get(json_url)
    response.raise_for_status()
    expected_results = json.loads(response.text)
except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching the JSON data: {e}")
    expected_results = {}

def process_people(file_url: str) -> list:

    pass

file_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
your_results = process_people(file_url)

if your_results == expected_results:
    print("Your results match the expected results.")
else:
    print("Your results do not match the expected results.")

Your results do not match the expected results.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---